# Jaqal to IR

In this tutorial we will examine Jaqal. [Jaqal](https://www.sandia.gov/quantum/quantum-information-sciences/projects/qscout-jaqal/) (Just Another Quantum Assembly Language) is the QASM built by Sandia Labs for the [QSCOUT](https://www.sandia.gov/quantum/quantum-information-sciences/projects/qscout/) trapped-ion backend.

## JaqalPaq

We will primarily be using [JaqalPaq](https://gitlab.com/jaqal/jaqalpaq), the python package for Jaqal meta-programming. For details about JaqalPaq installation, see their [guide](https://gitlab.com/jaqal/jaqalpaq).

## Basic Usage Example

We've provided the `Sxx_circuit.jaqal` file in this repository: to confirm that your installation is valid, run the following code cell. You should see an idealized result array from the noiseless emulator: 50% |00>, 50% |11>

In [1]:
pip install pygsti

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install daal==2021.4.0

  Using cached daal-2021.4.0-py2.py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.whl (189.9 MB)
  Using cached tbb-2021.6.0-py2.py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.whl (1.0 MB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2
ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy==1.21

Note: you may need to restart the kernel to use updated packages.


In [7]:
import jaqalpaq

In [11]:
from jaqalpaq.transpiler import 

ModuleNotFoundError: No module named 'jaqalpaq.transpiler'

In [12]:
pip install JaqalPaq-Extras

  Using cached JaqalPaq_extras-1.0.0-py2.py3-none-any.whl (47 kB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
from jaqalpaq.parser import parse_jaqal_file
from jaqalpaq.emulator import run_jaqal_circuit
from jaqalpaq.generator import generate_jaqal_program

JaqalCircuitObject = parse_jaqal_file("Sxx_circuit.jaqal")
JaqalCircuitResults = run_jaqal_circuit(JaqalCircuitObject)
print(f"Probabilities: {JaqalCircuitResults.subcircuits[0].probability_by_str}")
JaqalProgram = generate_jaqal_program(JaqalCircuitObject)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## Translating to Jaqal

Jaqal has built in capability to take circuits from a variety of widly used languages (Qiskit, Quil, pytket, Cirq) and turn them into Jaqal code which can run on QSCOUT. We will walk through the transpilation of our Bernstein-Vazirani circuit from Qiskit into `bernstein_vazirani.jaqal`.

In [5]:
from qiskit import QuantumCircuit


s = '011'   # the hidden binary string
n = 3 # number of bits used to represent s


# We need a circuit with n qubits, plus one auxiliary qubit
# We also need n classical bits to write the output to
qc = QuantumCircuit(n+1, n)

# Put auxiliary in the minus state |->
# We leave the other n qubits in just |0>
qc.h(n)
qc.z(n)

# Apply Hadamard gates to each of the n "main" qubits (excluding the auxiliary)
# before querying the oracle
for i in range(n):
    qc.h(i)
    
# Apply barrier
qc.barrier()

# Apply the inner-product oracle
s = s[::-1] # reverse s to fit qiskit's qubit ordering
for q in range(n):
    if s[q] == '0':
        qc.i(q)
    else:
        qc.cx(q, n)
        
# Apply barrier
qc.barrier()

# Apply Hadamard gates after querying the oracle
for i in range(n):
    qc.h(i)


# Measurement
for i in range(n):
    qc.measure(i, i)

qc.draw()

┌───┐      ░                 ░ ┌───┐┌─┐      
q_0: ┤ H ├──────░───■─────────────░─┤ H ├┤M├──────
     ├───┤      ░   │             ░ ├───┤└╥┘┌─┐   
q_1: ┤ H ├──────░───┼─────────■───░─┤ H ├─╫─┤M├───
     ├───┤      ░   │  ┌───┐  │   ░ ├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├──────░───┼──┤ I ├──┼───░─┤ H ├─╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐└───┘┌─┴─┐ ░ └───┘ ║  ║ └╥┘
q_3: ┤ H ├┤ Z ├─░─┤ X ├─────┤ X ├─░───────╫──╫──╫─
     └───┘└───┘ ░ └───┘     └───┘ ░       ║  ║  ║ 
c: 3/═════════════════════════════════════╩══╩══╩═
                                          0  1  2

## Translation to Jaqal

Translation from Qiskit to Jaqal occurs in two steps. First, the above circuit must be unrolled to QSCOUT's native gate set, composed of ion compatible instructions. Then, we can directly translate that circuit object into a Jaqal circuit object.

In [8]:
from qiskit.transpiler import PassManager
from jaqalpaq.transpilers.qiskit import ion_pass_manager, jaqal_circuit_from_qiskit_circuit

# ion_pass_manager provides a qiskit PassManager object which is supposed to 
# unroll a qiskit circuit to Jaqal gates, however it struggles
# with Qiskit 'I' gate
ion_qc = ion_pass_manager().run(qc)


ModuleNotFoundError: No module named 'jaqalpaq.transpilers'

In [ ]:
# In order to deal with I, we need to add a relation to the Equivalence Library
# of a pass called 'Unroll Custom Definitions'. To get at that pass:
pm = ion_pass_manager()
passes_array_dict_array = pm.passes()
passes_array_dict = passes_array_dict_array[0]
passes_array = passes_array_dict['passes']
unroll_custom_defs = passes_array[0]
print(dir(unroll_custom_defs))



In [ ]:
# Now, we get the equivalence library and add the relation
# I_Px (Jaqal Identity gate with same duration as Px) = I
equiv_lib = unroll_custom_defs._equiv_lib
#print(dir(equiv_lib))
#print(equiv_lib.__dict__.items())
print(equiv_lib.__dict__['_base'].__dict__.items())
# equiv_lib.add_equivalence()


#pm._equiv_lib.append()
#jaq_compat = pm.run(qc)

#jaqc = jaqal_circuit_from_qiskit_circuit(jaq_compat) #qc, qiskit_to_jaq_map)
#print(generate_jaqal_program(jaqc))